In [344]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

## 데이터 수집 및 전처리
### kosdaq

In [345]:
## kosdqp
## csv 파일로부터 재무제표 데이터를 불러옴
kosdaq1 = pd.read_csv("./공모전파일/코스닥데이터(연결재무제표).csv", encoding = "euc-kr")
kosdaq2 = pd.read_csv("./공모전파일/코스닥데이터(개별재무제표).csv", encoding = "euc-kr")
kosdaq1.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kosdaq2.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
## 결측치를 개별재무제표로 대체함
kosdaq = kosdaq1.fillna(kosdaq2)
## 해당년도 재무제표가 존재하지 않는 기업 해당년도 삭제
# kosdaq.isna().sum()
kosdaq.dropna(subset=["매출액증가율"], inplace = True)
# kosdaq.isna().sum()
# 상장일 이전에 회계년도 데이터 삭제
kosdaq['상장일'] = pd.to_datetime(kosdaq['상장일'])
kosdaq['회계년도'] = pd.to_datetime(kosdaq['회계년도'], format='%Y/%m')
kosdaq = kosdaq.loc[kosdaq['회계년도'] >= kosdaq['상장일']]
## kosdaq 외국계 기업 종목코드 제거
kosdaq = kosdaq[kosdaq['거래소코드'] < 900000]
## 2. 거래소코드 2개인 기업들 셀렉
# grouped = kosdaq.groupby('회사명')['거래소코드'].nunique()
# result = grouped[grouped > 1].index.tolist()
# print(result)
## 거래소코드 2개인 기업 확인하여 삭제
codes = [271400, 60540, 262760, 278380, 131390, 230400, 284610, 176560, 284420, 251960, 216280,25440, 229480]
kosdaq = kosdaq.loc[~(kosdaq['거래소코드'].isin(codes))]
## 증가율 0 인 데이터 평균값으로 대체
kosdaq['자기자본증가율'] = kosdaq['자기자본증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['자기자본증가율'].mean()
kosdaq['자기자본증가율'] = kosdaq['자기자본증가율'].fillna(mean_growth_rate)
kosdaq['순이익증가율'] = kosdaq['순이익증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['순이익증가율'].mean()
kosdaq['순이익증가율'] = kosdaq['순이익증가율'].fillna(mean_growth_rate)
kosdaq['매출액증가율'] = kosdaq['매출액증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['매출액증가율'].mean()
kosdaq['매출액증가율'] = kosdaq['매출액증가율'].fillna(mean_growth_rate)
kosdaq['총자본증가율'] = kosdaq['총자본증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['총자본증가율'].mean()
kosdaq['총자본증가율'] = kosdaq['총자본증가율'].fillna(mean_growth_rate)
kosdaq['유형자산증가율'] = kosdaq['유형자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['유형자산증가율'].mean()
kosdaq['유형자산증가율'] = kosdaq['유형자산증가율'].fillna(mean_growth_rate)
kosdaq['비유동자산증가율'] = kosdaq['비유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['비유동자산증가율'].mean()
kosdaq['비유동자산증가율'] = kosdaq['비유동자산증가율'].fillna(mean_growth_rate)
kosdaq['유동자산증가율'] = kosdaq['유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['유동자산증가율'].mean()
kosdaq['유동자산증가율'] = kosdaq['유동자산증가율'].fillna(mean_growth_rate)
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].replace(0, np.nan)
mean_growth_rate = kosdaq['재고자산증가율'].mean()
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].fillna(mean_growth_rate)
kosdaq = kosdaq.astype({"거래소코드" : "str"})
kosdaq = kosdaq.astype({"소속코드" : "str"})
kosdaq

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일
0,(주)CMG제약,58820,2018-12-01,20.05,4.39,6.16,113.940000,6.71,16.47,0.18,...,14.73,106.25,478.91,33.79,67.33,32.67,36.00,NaN,5,2001-08-31
1,(주)CMG제약,58820,2019-12-01,17.56,68.07,73.26,-49.690000,39.67,26.20,85.06,...,9.90,114.60,438.05,32.47,72.60,27.40,16.03,NaN,5,2001-08-31
2,(주)CMG제약,58820,2020-12-01,9.96,1.62,0.54,-92.950000,23.31,18.52,-3.06,...,8.71,108.22,475.13,26.40,86.14,13.86,1.26,NaN,5,2001-08-31
3,(주)CMG제약,58820,2021-12-01,7.78,1.95,1.97,1372.340000,2.61,49.25,-14.04,...,9.69,126.93,601.75,27.77,77.39,22.61,16.41,NaN,5,2001-08-31
4,(주)CMG제약,58820,2022-12-01,18.31,7.07,0.14,60.737581,22.79,6.10,7.64,...,7.47,106.98,537.68,19.38,104.24,-4.24,-2.44,NaN,5,2001-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7653,흥구석유(주),24060,2018-12-01,6.87,-5.89,0.99,14.780000,-0.36,-2.09,-21.19,...,9.80,19.56,3064.10,4.44,47.47,52.53,32.14,NaN,5,1994-12-07
7654,흥구석유(주),24060,2019-12-01,-9.35,11.24,9.31,249.940000,-7.00,-2.63,80.74,...,18.02,42.99,3489.21,10.00,22.19,77.81,55.02,NaN,5,1994-12-07
7655,흥구석유(주),24060,2020-12-01,-23.78,0.07,3.38,-42.220000,-13.92,-8.21,22.41,...,12.27,34.04,3280.80,8.94,30.41,69.59,46.65,NaN,5,1994-12-07
7656,흥구석유(주),24060,2021-12-01,15.12,-1.28,-0.88,-70.810000,0.63,0.18,-4.24,...,7.01,19.15,1308.44,4.38,53.22,46.78,24.13,NaN,5,1994-12-07


In [346]:
# 윈저라이징으로 양쪽 이상치 처리
kosdaq['매출액증가율']

q5 = np.percentile(kosdaq['매출액증가율'], 5)
q95 = np.percentile(kosdaq['매출액증가율'], 95)

outliers = kosdaq['매출액증가율'][(kosdaq['매출액증가율'] < q5) | (kosdaq['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치 :", outliers)

하위 5%: -40.28
상위 5%: 78.68599999999996
이상치 : 7        84.57
26      -50.47
38      301.12
46      -45.85
47      226.35
         ...  
7636     91.21
7638    -57.02
7639    -42.66
7642     80.87
7646    603.96
Name: 매출액증가율, Length: 669, dtype: float64


In [347]:
# 윈저라이징 함수 만들기
def windsorize(data, lower_percentile=0.05, upper_percentile=0.95):
    if isinstance(data, pd.Series):
        data = pd.DataFrame(data)
    numeric_df = data.select_dtypes(include=np.number)
    lower_bound = np.percentile(numeric_df, lower_percentile * 100)
    upper_bound = np.percentile(numeric_df, upper_percentile * 100)
    windsorized_data = np.where(numeric_df < lower_bound, lower_bound, numeric_df)
    windsorized_data = np.where(windsorized_data > upper_bound, upper_bound, windsorized_data)
    return windsorized_data

In [348]:
for i in range(3, 50, 1):
    kosdaq[kosdaq.columns[i]] = windsorize(kosdaq[kosdaq.columns[i]])

In [349]:
# 전후 비교
kosdaq['매출액증가율']

q5 = np.percentile(kosdaq['매출액증가율'], 5)
q95 = np.percentile(kosdaq['매출액증가율'], 95)

outliers = kosdaq['매출액증가율'][(kosdaq['매출액증가율'] < q5) | (kosdaq['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

하위 5%: -40.28
상위 5%: 78.65719999999999
이상치: 7       78.686
38      78.686
47      78.686
50      78.686
78      78.686
         ...  
7548    78.686
7587    78.686
7636    78.686
7642    78.686
7646    78.686
Name: 매출액증가율, Length: 335, dtype: float64


In [350]:
# standard 스케일링
numeric_cols = kosdaq.select_dtypes(include=np.number).columns
scaler = StandardScaler()
kosdaq[numeric_cols] = scaler.fit_transform(kosdaq[numeric_cols])

print(kosdaq.head())

        회사명  거래소코드       회계년도    매출액증가율    총자본증가율   자기자본증가율    순이익증가율   
0  (주)CMG제약  58820 2018-12-01  0.267103 -0.329883 -0.251115  1.124574  \
1  (주)CMG제약  58820 2019-12-01  0.182738  2.461924  2.084707 -1.427800   
2  (주)CMG제약  58820 2020-12-01 -0.074763 -0.451324 -0.446753 -1.983355   
3  (주)CMG제약  58820 2021-12-01 -0.148625 -0.436856 -0.396974  2.546591   
4  (주)CMG제약  58820 2022-12-01  0.208149 -0.212389 -0.460678  0.294699   

    유형자산증가율  비유동자산증가율   유동자산증가율  ...   총자본투자효율    설비투자효율    기계투자효율     부가가치율   
0 -0.273520  0.036829 -0.406337  ... -0.046738 -0.176940 -0.346626  0.465836  \
1  0.606500  0.344726  2.120575  ... -0.394861 -0.148162 -0.352061  0.421143   
2  0.169694  0.101700 -0.502793  ... -0.480630 -0.170151 -0.347129  0.215627   
3 -0.382989  1.074120 -0.829672  ... -0.409996 -0.105667 -0.330288  0.262012   
4  0.155810 -0.291319 -0.184250  ... -0.570003 -0.174424 -0.338809 -0.022055   

    노동소득분배율     자본분배율     이윤분배율  상장폐지일  소속코드        상장일  
0  0.052136  0.247429 

In [351]:
# 체크하는 법
kosdaq[numeric_cols].mean()

매출액증가율       1.697579e-17
총자본증가율       1.697579e-17
자기자본증가율     -1.909776e-17
순이익증가율       1.114036e-17
유형자산증가율     -2.228072e-17
비유동자산증가율     7.639104e-17
유동자산증가율      6.365920e-18
재고자산증가율     -1.644529e-17
유동비율        -1.273184e-16
당좌비율        -1.697579e-16
현금비율         1.188305e-16
순운전자본비율     -6.790314e-17
부채비율         1.612700e-16
자기자본배율       5.517130e-17
비유동비율        3.098081e-16
유동자산구성비율    -2.376610e-16
자기자본구성비율     4.519803e-16
매출액총이익률     -5.517130e-17
매출액순이익률      5.941525e-17
총자본사업이익률    -1.273184e-17
총자본순이익률      8.487893e-18
자기자본순이익률     1.060987e-17
1주당순이익(원)    1.273184e-17
1주당매출액(원)   -8.912288e-17
총자본회전률       2.291731e-16
자기자본회전률      8.063498e-17
비유동자산회전률     1.007937e-16
재고자산회전률     -2.758565e-17
매출채권회전률     -4.243946e-17
매입채무회전률      2.121973e-18
매입채무회전기간    -1.156475e-16
재고자산회전기간    -7.267758e-17
매출채권회전기간     2.546368e-17
1회전기간       -1.856727e-17
운전가본회전률     -2.970763e-17
순운전자본회전률    -5.092736e-17
유형자산회전율      8.063498e-17
유동자산회전률      8.487893e-18
당좌자산회전률     

In [352]:
kosdaq[numeric_cols].std()

매출액증가율       1.000075
총자본증가율       1.000075
자기자본증가율      1.000075
순이익증가율       1.000075
유형자산증가율      1.000075
비유동자산증가율     1.000075
유동자산증가율      1.000075
재고자산증가율      1.000075
유동비율         1.000075
당좌비율         1.000075
현금비율         1.000075
순운전자본비율      1.000075
부채비율         1.000075
자기자본배율       1.000075
비유동비율        1.000075
유동자산구성비율     1.000075
자기자본구성비율     1.000075
매출액총이익률      1.000075
매출액순이익률      1.000075
총자본사업이익률     1.000075
총자본순이익률      1.000075
자기자본순이익률     1.000075
1주당순이익(원)    1.000075
1주당매출액(원)    1.000075
총자본회전률       1.000075
자기자본회전률      1.000075
비유동자산회전률     1.000075
재고자산회전률      1.000075
매출채권회전률      1.000075
매입채무회전률      1.000075
매입채무회전기간     1.000075
재고자산회전기간     1.000075
매출채권회전기간     1.000075
1회전기간        1.000075
운전가본회전률      1.000075
순운전자본회전률     1.000075
유형자산회전율      1.000075
유동자산회전률      1.000075
당좌자산회전률      1.000075
부가가치(백만원)    1.000075
총자본투자효율      1.000075
설비투자효율       1.000075
기계투자효율       1.000075
부가가치율        1.000075
노동소득분배율      1.000075
자본분배율     

### kospi

In [353]:
kospi1 = pd.read_csv("./공모전파일/코스피데이터(연결재무제표).csv", encoding = "euc-kr")
kospi2 = pd.read_csv("./공모전파일/코스피데이터(개별재무제표).csv", encoding = "euc-kr")
kospi1.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kospi2.columns = ["회사명", "거래소코드", "회계년도", "매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율", "상장폐지일", "소속코드", "상장일"]
kospi = kospi1.fillna(kospi2)
kospi.dropna(subset=["매출액증가율"], inplace = True)
kospi['상장일'] = pd.to_datetime(kospi['상장일'])
kospi['회계년도'] = pd.to_datetime(kospi['회계년도'], format='%Y/%m')
kospi = kospi.loc[kospi['회계년도'] >= kospi['상장일']]
kospi = kospi[kospi['거래소코드'] < 900000]
kospi['자기자본증가율'] = kospi['자기자본증가율'].replace(0, np.nan)
mean_growth_rate = kospi['자기자본증가율'].mean()
kospi['자기자본증가율'] = kospi['자기자본증가율'].fillna(mean_growth_rate)
kospi['순이익증가율'] = kospi['순이익증가율'].replace(0, np.nan)
mean_growth_rate = kospi['순이익증가율'].mean()
kospi['순이익증가율'] = kospi['순이익증가율'].fillna(mean_growth_rate)
kospi['유동자산증가율'] = kospi['유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['유동자산증가율'].mean()
kospi['유동자산증가율'] = kospi['유동자산증가율'].fillna(mean_growth_rate)
kospi['재고자산증가율'] = kospi['재고자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['재고자산증가율'].mean()
kospi['재고자산증가율'] = kospi['재고자산증가율'].fillna(mean_growth_rate)
kospi['유형자산증가율'] = kospi['유형자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['유형자산증가율'].mean()
kospi['유형자산증가율'] = kospi['유형자산증가율'].fillna(mean_growth_rate)
kospi['비유동자산증가율'] = kospi['비유동자산증가율'].replace(0, np.nan)
mean_growth_rate = kospi['비유동자산증가율'].mean()
kospi['비유동자산증가율'] = kospi['비유동자산증가율'].fillna(mean_growth_rate)
kospi['매출액증가율'] = kospi['매출액증가율'].replace(0, np.nan)
mean_growth_rate = kospi['매출액증가율'].mean()
kospi['매출액증가율'] = kospi['매출액증가율'].fillna(mean_growth_rate)
kospi['총자본증가율'] = kospi['총자본증가율'].replace(0, np.nan)
mean_growth_rate = kospi['총자본증가율'].mean()
kospi['총자본증가율'] = kospi['총자본증가율'].fillna(mean_growth_rate)
kospi = kospi.astype({"거래소코드" : "str"})
kospi = kospi.astype({"소속코드" : "str"})
kospi

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일
0,(주)DB하이텍,990,2018-12-01,-1.54,7.30,22.510000,-21.190000,7.56,6.69,8.40,...,28.98,61.13,1.409500e+02,46.18,42.92,57.08,28.09,NaN,1,1975-12-12
1,(주)DB하이텍,990,2019-12-01,20.64,2.10,15.450000,20.510000,3.18,-3.00,11.19,...,34.83,67.30,1.352100e+02,46.97,38.21,61.79,27.59,NaN,1,1975-12-12
2,(주)DB하이텍,990,2020-12-01,15.92,8.24,23.260000,58.670000,9.42,8.56,7.73,...,41.06,83.62,1.773200e+02,51.71,32.58,67.42,34.31,NaN,1,1975-12-12
3,(주)DB하이텍,990,2021-12-01,29.78,31.51,39.200000,90.870000,-2.50,2.14,77.51,...,48.06,134.52,2.847200e+02,61.33,24.03,75.97,42.54,NaN,1,1975-12-12
4,(주)DB하이텍,990,2022-12-01,37.92,36.95,48.120000,76.350000,12.18,8.19,62.88,...,50.35,189.56,3.855200e+02,63.80,20.01,79.99,52.29,NaN,1,1975-12-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,흥아해운(주),3280,2018-12-01,-9.87,-5.33,-37.420000,273.538166,-12.07,-5.17,-6.33,...,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12,NaN,1,1976-06-29
3689,흥아해운(주),3280,2019-12-01,-86.45,-46.38,-57.730000,273.538166,-51.51,-51.51,-13.42,...,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23,NaN,1,1976-06-29
3690,흥아해운(주),3280,2020-12-01,-12.95,-33.46,-185.010000,273.538166,-24.76,-32.47,-37.01,...,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00,NaN,1,1976-06-29
3691,흥아해운(주),3280,2021-12-01,-8.11,-13.77,13.275779,273.538166,-20.91,-15.66,-6.48,...,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04,NaN,1,1976-06-29


In [354]:
for i in range(3, 50, 1):
    kospi[kospi.columns[i]] = windsorize(kospi[kospi.columns[i]])

In [355]:
# 전후 비교
kospi['매출액증가율']

q5 = np.percentile(kospi['매출액증가율'], 5)
q95 = np.percentile(kospi['매출액증가율'], 95)

outliers = kospi['매출액증가율'][(kospi['매출액증가율'] < q5) | (kospi['매출액증가율'] > q95)]

print("하위 5%:", q5)
print("상위 5%:", q95)
print("이상치:", outliers)

하위 5%: -23.2224
상위 5%: 48.9036
이상치: 5      -23.244
16     -23.244
17     -23.244
18      48.906
23      48.906
         ...  
3679    48.906
3681    48.906
3684    48.906
3689   -23.244
3692    48.906
Name: 매출액증가율, Length: 360, dtype: float64


In [356]:
# standard 스케일링
numeric_cols = kospi.select_dtypes(include=np.number).columns
scaler = StandardScaler()
kospi[numeric_cols] = scaler.fit_transform(kospi[numeric_cols])

print(kospi.head())

        회사명 거래소코드       회계년도    매출액증가율    총자본증가율   자기자본증가율    순이익증가율   
0  (주)DB하이텍   990 2018-12-01 -0.545082  0.029407  1.056604 -0.950430  \
1  (주)DB하이텍   990 2019-12-01  0.696357 -0.393348  0.580566 -0.662053   
2  (주)DB하이텍   990 2020-12-01  0.432174  0.105828  1.107175 -0.398156   
3  (주)DB하이텍   990 2021-12-01  1.207933  1.997657  2.181970 -0.175476   
4  (주)DB하이텍   990 2022-12-01  1.663538  2.439924  2.677024 -0.275890   

    유형자산증가율  비유동자산증가율   유동자산증가율  ...   총자본투자효율    설비투자효율    기계투자효율     부가가치율   
0  0.106446 -0.064104 -0.024497  ...  1.229743 -0.307570 -0.359919  1.901851  \
1 -0.150118 -0.690984  0.115852  ...  1.846057 -0.249279 -0.360986  1.964812   
2  0.215398  0.056872 -0.058201  ...  2.180658 -0.095095 -0.353156  2.255392   
3 -0.482831 -0.358459  2.395941  ...  2.180658  0.385785 -0.333188  2.255392   
4  0.377069  0.032936  2.395941  ...  2.180658  0.905778 -0.314447  2.255392   

    노동소득분배율     자본분배율     이윤분배율  상장폐지일  소속코드        상장일  
0 -0.514174  0.743839  0.629

In [357]:
# kospi 상장폐지 기업
kospi_sp = kospi.loc[~(kospi["상장폐지일"].isna())]
kospi_sp["회사명"].unique()

array(['(주)동성화학', '(주)부산도시가스', '(주)사조해표', '(주)신한', '(주)엔에스쇼핑',
       '(주)폴루스바이오팜', '(주)한화갤러리아타임월드', '고려개발(주)', '두산건설(주)', '롯데푸드(주)',
       '알보젠코리아(주)', '에스비에스미디어홀딩스(주)', '웅진에너지(주)', '코리아오토글라스(주)',
       '코오롱머티리얼(주)', '한국제지(주)'], dtype=object)

In [358]:
# kosdaq 상장폐지 기업
kosdaq_sp = kosdaq.loc[~(kosdaq["상장폐지일"].isna())]
kosdaq_sp["회사명"].unique()

array(['(주)금빛', '(주)녹십자셀', '(주)뉴로스', '(주)데코앤이', '(주)드림티엔터테인먼트', '(주)레드로버',
       '(주)맘스터치앤컴퍼니', '(주)맥스로텍', '(주)모다', '(주)미래SCI', '(주)미래테크놀로지',
       '(주)바이오빌', '(주)세영디앤씨', '(주)소리바다', '(주)썬테크놀로지스', '(주)씨엠에스에듀',
       '(주)아이엠텍', '(주)에스마크', '(주)에스에이티', '(주)에스에프씨', '(주)에스제이케이',
       '(주)에이치디', '(주)에이팸', '(주)에프티이앤이', '(주)엘엑스세미콘', '(주)엘피케이로보틱스',
       '(주)엠씨넥스', '(주)연이비앤티', '(주)원익피앤이', '(주)이매진아시아', '(주)이십일스토어',
       '(주)이엠네트웍스', '(주)인터파크', '(주)제낙스', '(주)제이콘텐트리', '(주)제이테크놀로지',
       '(주)지에스홈쇼핑', '(주)지유온', '(주)카이노스메드', '(주)켐온', '(주)코너스톤네트웍스',
       '(주)코리아센터', '(주)쿠첸', '(주)티씨엠생명과학', '(주)티에스트릴리온', '(주)파인넥스',
       '(주)파티게임즈', '(주)팍스넷', '(주)포스코케미칼', '(주)포인트엔지니어링', '(주)폭스소프트',
       '(주)퓨전', '(주)피앤텔', '(주)한국비엔씨', '(주)한국아트라스비엑스', '(주)한컴지엠디', '(주)한프',
       '(주)행남사', '(주)화신테크', '(주)화진', '넥슨지티(주)', '다이노나(주)', '디에스티(주)',
       '럭슬(주)', '미래오토스', '에스에이치엔엘(주)', '에스지에이클라우드서비스(주)', '에스케이 머티리얼즈(주)',
       '에스케이씨솔믹스(주)', '에이아이비트(주)', '원텍(주)', '이엘케이(주)', '자안바이오(주)',
       '지스마트글로벌(주)', '참존글로벌(

#### krx 상장폐지 현황 데이터의 상장폐지사유와 비교하여 부실여부 판단

In [359]:
# 20180101 ~ 20221231 상장폐지종목 데이터
# sp = pd.read_excel("./상장폐지기업.xlsx")
# cond = (sp["증권구분"] == "주권") & (sp["주식종류"] == "보통주") & (sp['시장구분'] != 'KONEX')
# sp = sp[cond]
# sp = sp.drop('번호', axis=1)
# sp.to_excel("delist.xlsx")

In [360]:
# sp.loc[sp['종목명'].isin(['동성화학', '부산도시가스', '사조해표', '신한', '엔에스쇼핑',
#        '폴루스바이오팜', '한화갤러리아타임월드', '고려개발', '두산건설', '롯데푸드',
#        '알보젠코리아', '에스비에스미디어홀딩스', '웅진에너지', '코리아오토글라스',
#        '코오롱머티리얼', '한국제지', 'SBS미디어홀딩스', '부산가스'])]

In [361]:
# # krx 데이터에서 kospi 상장폐지 종목 확인
# sp.loc[sp['종목명'].isin(['동성화학', '부산도시가스', '사조해표', '신한', '엔에스쇼핑',
#        '폴루스바이오팜', '한화갤러리아타임월드', '고려개발', '두산건설', '롯데푸드',
#        '알보젠코리아', '에스비에스미디어홀딩스', '웅진에너지', '코리아오토글라스',
#        '코오롱머티리얼', '한국제지', 'SBS미디어홀딩스', '부산가스'])]

In [362]:
# kospi 중 부실, 정상 기업 추출
kospi_bs = kospi.loc[kospi['회사명'].isin(['(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', 
                                        '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', 
                                        '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'])]
kospi_js = kospi.loc[~kospi['회사명'].isin(['(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', 
                                         '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', 
                                         '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'])]

In [363]:
# # krx 데이터에서 kosdaq 상장폐지 종목 확인
# sp.loc[sp['종목명'].isin(['금빛', '녹십자셀', '뉴로스', '데코앤이', '드림티엔터테인먼트', '레드로버',
#        '맘스터치앤컴퍼니', '맥스로텍', '모다', '미래SCI', '미래테크놀로지',
#        '바이오빌', '세영디앤씨', '소리바다', '썬테크놀로지스', '씨엠에스에듀',
#        '아이엠텍', '에스마크', '에스에이티', '에스에프씨', '에스제이케이',
#        '에이치디', '에이팸', '에프티이앤이', '엘엑스세미콘', '엘피케이로보틱스',
#        '엠씨넥스', '연이비앤티', '원익피앤이', '이매진아시아', '이십일스토어',
#        '이엠네트웍스', '인터파크', '제낙스', '제이콘텐트리', '제이테크놀로지',
#        '지에스홈쇼핑', '지유온', '카이노스메드', '켐온', '코너스톤네트웍스',
#        '코리아센터', '쿠첸', '티씨엠생명과학', '티에스트릴리온', '파인넥스',
#        '파티게임즈', '팍스넷', '포스코케미칼', '포인트엔지니어링', '폭스소프트',
#        '퓨전', '피앤텔', '한국비엔씨', '한국아트라스비엑스', '한컴지엠디', '한프',
#        '행남사', '화신테크', '화진', '넥슨지티', '다이노나', '디에스티',
#        '럭슬', '미래오토스', '에스에이치엔엘', '에스지에이클라우드서비스', '에스케이 머티리얼즈',
#        '에스케이씨솔믹스', '에이아이비트', '원텍', '이엘케이', '자안바이오',
#        '지스마트글로벌', '참존글로벌', '케이맥', '피아이첨단소재', '한일네트웍스',
#        '현대정보기술', '현진소재'])]

In [364]:
# kosdaq 중 부실, 정상 기업 추출
kosdaq_bs = kosdaq.loc[kosdaq['회사명'].isin(['(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'])]

kosdaq_js = kosdaq.loc[~kosdaq['회사명'].isin(['(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'])]

In [365]:
# kospi 데이터에 "부실여부" 파생변수 삽입
kospibs = '(주)동성화학', '(주)사조해표', '(주)신한', '(주)폴루스바이오팜', '고려개발(주)', '롯데푸드(주)', '에스비에스미디어홀딩스(주)', '웅진에너지(주)', '코리아오토글라스(주)', '한국제지(주)'
kospi["부실여부"] = kospi['회사명'].apply(lambda x: 1 if x in kospibs else 0)
kospi.head()

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일,부실여부
0,(주)DB하이텍,990,2018-12-01,-0.545082,0.029407,1.056604,-0.950430,0.106446,-0.064104,-0.024497,...,-0.307570,-0.359919,1.901851,-0.514174,0.743839,0.629616,NaN,1,1975-12-12,0
1,(주)DB하이텍,990,2019-12-01,0.696357,-0.393348,0.580566,-0.662053,-0.150118,-0.690984,0.115852,...,-0.249279,-0.360986,1.964812,-0.681920,0.923410,0.613246,NaN,1,1975-12-12,0
2,(주)DB하이텍,990,2020-12-01,0.432174,0.105828,1.107175,-0.398156,0.215398,0.056872,-0.058201,...,-0.095095,-0.353156,2.255392,-0.882431,1.138056,0.833252,NaN,1,1975-12-12,0
3,(주)DB하이텍,990,2021-12-01,1.207933,1.997657,2.181970,-0.175476,-0.482831,-0.358459,2.395941,...,0.385785,-0.333188,2.255392,-1.186938,1.464028,1.102692,NaN,1,1975-12-12,0
4,(주)DB하이텍,990,2022-12-01,1.663538,2.439924,2.677024,-0.275890,0.377069,0.032936,2.395941,...,0.905778,-0.314447,2.255392,-1.330109,1.468603,1.404545,NaN,1,1975-12-12,0


In [366]:
# kosdaq 데이터에 "부실여부" 파생변수 삽입
kosdaqbs = '(주)썬테크놀로지스', '참존글로벌(주)', '(주)레드로버', '(주)뉴로스', '(주)연이비앤티', 
'(주)소리바다', '에스에이치엔엘(주)', '(주)세영디앤씨', '현진소재(주)', '(주)한프',
'지스마트글로벌(주)', '자안바이오(주)', '(주)이엠네트웍스', '(주)코너스톤네트웍스', 
'에이아이비트(주)', '(주)지유온', '(주)맥스로텍', '(주)에스제이케이', '디에스티(주)', 
'(주)팍스넷', '럭슬(주)', '(주)제낙스', '(주)아이엠텍', '(주)퓨전', '(주)행남사',
'(주)에이팸', '(주)에이치디', '(주)미래SCI', '(주)이매진아시아', '(주)파티게임즈', 
'(주)바이오빌', '(주)모다', '(주)화진', '(주)에스에프씨', '이엘케이(주)', '(주)에스마크',
'(주)피앤텔', '(주)파인넥스', '(주)제이테크놀로지', '(주)데코앤이', '(주)에프티이앤이',
'(주)이십일스토어', '(주)화신테크', '미래오토스', '에스지에이클라우드서비스(주)'
kosdaq["부실여부"] = kosdaq['회사명'].apply(lambda x: 1 if x in kosdaqbs else 0)
kosdaq.head()

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일,부실여부
0,(주)CMG제약,58820,2018-12-01,0.267103,-0.329883,-0.251115,1.124574,-0.273520,0.036829,-0.406337,...,-0.176940,-0.346626,0.465836,0.052136,0.247429,0.637277,NaN,5,2001-08-31,0
1,(주)CMG제약,58820,2019-12-01,0.182738,2.461924,2.084707,-1.427800,0.606500,0.344726,2.120575,...,-0.148162,-0.352061,0.421143,0.151421,0.134455,0.331270,NaN,5,2001-08-31,0
2,(주)CMG제약,58820,2020-12-01,-0.074763,-0.451324,-0.446753,-1.983355,0.169694,0.101700,-0.502793,...,-0.170151,-0.347129,0.215627,0.406509,-0.155806,0.104944,NaN,5,2001-08-31,0
3,(주)CMG제약,58820,2021-12-01,-0.148625,-0.436856,-0.396974,2.546591,-0.382989,1.074120,-0.829672,...,-0.105667,-0.330288,0.262012,0.241663,0.031770,0.337093,NaN,5,2001-08-31,0
4,(주)CMG제약,58820,2022-12-01,0.208149,-0.212389,-0.460678,0.294699,0.155810,-0.291319,-0.184250,...,-0.174424,-0.338809,-0.022055,0.747506,-0.543821,0.048248,NaN,5,2001-08-31,0


In [367]:
# kospi, kosdaq 데이터 병합
data = pd.concat([kospi, kosdaq], axis = 0)

In [368]:
data

,회사명,거래소코드,회계년도,매출액증가율,총자본증가율,자기자본증가율,순이익증가율,유형자산증가율,비유동자산증가율,유동자산증가율,...,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,상장폐지일,소속코드,상장일,부실여부
0,(주)DB하이텍,990,2018-12-01,-0.545082,0.029407,1.056604,-0.950430,0.106446,-0.064104,-0.024497,...,-0.307570,-0.359919,1.901851,-0.514174,0.743839,0.629616,NaN,1,1975-12-12,0
1,(주)DB하이텍,990,2019-12-01,0.696357,-0.393348,0.580566,-0.662053,-0.150118,-0.690984,0.115852,...,-0.249279,-0.360986,1.964812,-0.681920,0.923410,0.613246,NaN,1,1975-12-12,0
2,(주)DB하이텍,990,2020-12-01,0.432174,0.105828,1.107175,-0.398156,0.215398,0.056872,-0.058201,...,-0.095095,-0.353156,2.255392,-0.882431,1.138056,0.833252,NaN,1,1975-12-12,0
3,(주)DB하이텍,990,2021-12-01,1.207933,1.997657,2.181970,-0.175476,-0.482831,-0.358459,2.395941,...,0.385785,-0.333188,2.255392,-1.186938,1.464028,1.102692,NaN,1,1975-12-12,0
4,(주)DB하이텍,990,2022-12-01,1.663538,2.439924,2.677024,-0.275890,0.377069,0.032936,2.395941,...,0.905778,-0.314447,2.255392,-1.330109,1.468603,1.404545,NaN,1,1975-12-12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7653,흥구석유(주),24060,2018-12-01,-0.179457,-0.780571,-0.431088,-0.422168,-0.462287,-0.550483,-1.042530,...,-0.475717,-0.002774,-0.527890,-0.322018,0.673174,0.578129,NaN,5,1994-12-07,0
7654,흥구석유(주),24060,2019-12-01,-0.729018,-0.029571,-0.141460,2.546591,-0.639573,-0.567571,1.991966,...,-0.394965,0.053769,-0.339640,-0.798283,1.110366,0.909726,NaN,5,1994-12-07,0
7655,흥구석유(주),24060,2020-12-01,-1.217930,-0.519277,-0.347890,-1.311280,-0.824334,-0.744145,0.255459,...,-0.425811,0.026049,-0.375530,-0.643421,1.038894,0.800471,NaN,5,1994-12-07,0
7656,흥구석유(주),24060,2021-12-01,0.100066,-0.578463,-0.496185,-1.757240,-0.435854,-0.478651,-0.537922,...,-0.477130,-0.236292,-0.529921,-0.213690,0.549909,0.455389,NaN,5,1994-12-07,0


## 피쳐선정

In [369]:
# lasso

data_feature = data[["매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", 
          "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", 
          "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", 
          "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", 
          "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", 
          "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", 
          "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", 
          "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율"]]

feature = data_feature
target = data['부실여부']

lasso = SelectFromModel(LogisticRegression(C=0.2, penalty='l1',solver='liblinear'))
lasso.fit(feature, target)
lasso_support = lasso.get_support()
lr_feature = feature.loc[:,lasso_support].columns.tolist()
lr_feature

['매출액증가율',
 '자기자본증가율',
 '순이익증가율',
 '유형자산증가율',
 '비유동자산증가율',
 '재고자산증가율',
 '순운전자본비율',
 '매출액순이익률',
 '총자본사업이익률',
 '1주당매출액(원)',
 '재고자산회전률',
 '매출채권회전률',
 '매입채무회전률',
 '매입채무회전기간',
 '매출채권회전기간',
 '1회전기간',
 '운전가본회전률',
 '유동자산회전률',
 '부가가치(백만원)',
 '설비투자효율',
 '부가가치율',
 '노동소득분배율',
 '자본분배율']

In [370]:
# selectKBest

data_feature = data[["매출액증가율", "총자본증가율", "자기자본증가율", "순이익증가율", "유형자산증가율", "비유동자산증가율", 
          "유동자산증가율", "재고자산증가율", "유동비율", "당좌비율", "현금비율", "순운전자본비율", "부채비율", 
          "자기자본배율", "비유동비율", "유동자산구성비율", "자기자본구성비율", "매출액총이익률", "매출액순이익률", 
          "총자본사업이익률", "총자본순이익률", "자기자본순이익률", "1주당순이익(원)", "1주당매출액(원)", "총자본회전률", 
          "자기자본회전률", "비유동자산회전률", "재고자산회전률", "매출채권회전률", "매입채무회전률", "매입채무회전기간", 
          "재고자산회전기간", "매출채권회전기간", "1회전기간", "운전가본회전률", "순운전자본회전률", "유형자산회전율", 
          "유동자산회전률", "당좌자산회전률", "부가가치(백만원)", "총자본투자효율", "설비투자효율", "기계투자효율", 
          "부가가치율", "노동소득분배율", "자본분배율", "이윤분배율"]]

# 독립변수와 종속변수를 분리합니다.
X = data_feature
y = data['부실여부']

# f 검정 방법을 사용해 피처를 선택합니다.'k=선정개수'
selector = SelectKBest(score_func=f_classif, k=20)
X_new = selector.fit_transform(X, y)

# 선택된 피처를 출력합니다.
print(X.columns[selector.get_support()])

Index(['매출액증가율', '총자본증가율', '유형자산증가율', '비유동자산증가율', '현금비율', '순운전자본비율',
       '자기자본구성비율', '매출액순이익률', '총자본사업이익률', '총자본순이익률', '자기자본순이익률', '1주당순이익(원)',
       '매입채무회전기간', '매출채권회전기간', '순운전자본회전률', '부가가치(백만원)', '총자본투자효율', '부가가치율',
       '노동소득분배율', '자본분배율'],
      dtype='object')


## 모델링 및 성능평가

In [371]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 학습용 데이터와 테스트용 데이터 분할
train = data[data['회계년도'] < '2021-01-01']
test = data[data['회계년도'] >= '2021-01-01']

# 학습용 데이터 전처리
X_train = train[["현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
y_train = train['부실여부']

# 테스트용 데이터 전처리
X_test = test[["현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
y_test = test['부실여부']

# 랜덤 포레스트 모델 학습
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf.predict(X_test)

# 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# 평가 지표를 리스트로 저장
eval_metrics = ['Accuracy']
eval_values = [accuracy]

# 리스트를 데이터프레임으로 변환하여 출력
eval_df = pd.DataFrame({'Metric': eval_metrics, 'Value': eval_values})
eval_df


/Users/noseohyeong/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.998845


In [372]:
# 로지스틱
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 모델 선택 및 훈련
model = LogisticRegression()
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# 평가 지표를 리스트로 저장
evalmetrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC Score']
evalvalues = [accuracy, precision, recall, f1, roc_auc]

# 리스트를 데이터프레임으로 변환하여 출력
evaldf = pd.DataFrame({'Metric': evalmetrics, 'Value': evalvalues})
evaldf


/Users/noseohyeong/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.998845
1,Precision,0.000000
2,Recall,0.000000
3,F1 Score,0.000000
4,ROC AUC Score,0.500000


## 지수화
### 코스피200 대형주 50% 그 외 50% 랜덤추출하여 60종목 편입

In [373]:
# 코스피200 1분기, 반기 데이터 불러오기
df1 = pd.read_excel("./공모전파일/2020코스피200_1분기.xlsx")
df2 = pd.read_excel("./공모전파일/2020코스피200_반기.xlsx")
Scale = pd.read_csv("./공모전파일/기업규모코드.csv", encoding = "euc-kr")

In [374]:
# 데이터 프레임 크기 맞추기
common_companies = pd.Series(list(set(df1['회사명']) & set(df2['회사명']) & set(Scale['회사명'])))

df1 = df1[df1['회사명'].isin(common_companies)]
df2 = df2[df2['회사명'].isin(common_companies)]
Scale = Scale[Scale['회사명'].isin(common_companies)]


In [375]:
# 1분기 인덱스 리셋
df1 = df1.reset_index(drop=True)

In [376]:
# 반기 인덱스 리셋
df2 = df2.reset_index(drop=True)

In [377]:
# 1분기와 반기 데이터에 기업규모코드 추가
df1['기업규모코드'] = Scale['기업규모코드']
df2['기업규모코드'] = Scale['기업규모코드']

In [378]:
# 컬럼 이름 변경
df1.columns = ['회사명', '거래소코드', '회계년도', '자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익', '기업규모코드']
df2.columns = ['회사명', '거래소코드', '회계년도', '자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익', '기업규모코드']

In [379]:
# 컬럼 추출
df_1 = df1[['자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익']]
df_2 = df2[['자산', '현금및현금성자산', '부채', '유동자산', '유동부채', '원재료', '선급공사비,외주비', '매출액', '법인세비용차감전순이익', '법인세비용', '당기순이익']]

In [380]:
# 2분기 데이터 추출(반기 - 1분기)
df_2 = df_2 - df_1

In [381]:
# 필요한 컬럼만 추출
d1 = pd.concat([df1[['회사명', '거래소코드', '회계년도', '기업규모코드']], df_1], axis=1)
d2 = pd.concat([df2[['회사명', '거래소코드', '회계년도', '기업규모코드']], df_2], axis=1)

In [382]:
# 셀렉된 피처 컬럼 생성(1분기)
d1['현금비율'] = (d1['현금및현금성자산'] / d1['부채']) * 100
d1['순운전자본비율'] = ((d1['유동자산'] - d1['유동부채']) / d1['매출액']) * 100
d1['매출액순이익률'] = ((d1['법인세비용차감전순이익'] - d1['법인세비용']) / d1['매출액']) * 100
d1['총자본사업이익률'] = (d1['당기순이익'] / (d1['자산'] - d1['부채'])) * 100
d1['부가가치율'] = ((d1['매출액'] - d1['원재료'] - d1['선급공사비,외주비']) / d1['매출액']) * 100

In [383]:
# 셀렉된 피처 컬럼 생성(2분기)
d2['현금비율'] = (d2['현금및현금성자산'] / d2['부채']) * 100
d2['순운전자본비율'] = ((d2['유동자산'] - d2['유동부채']) / d2['매출액']) * 100
d2['매출액순이익률'] = ((d2['법인세비용차감전순이익'] - d2['법인세비용']) / d2['매출액']) * 100
d2['총자본사업이익률'] = (d2['당기순이익'] / (d2['자산'] - d2['부채'])) * 100
d2['부가가치율'] = ((d2['매출액'] - d2['원재료'] - d2['선급공사비,외주비']) / d2['매출액']) * 100

In [384]:
# 필요한 컬럼만 추출
d1 = d1[['회사명', '거래소코드', '회계년도', '기업규모코드', "현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]
d2 = d2[['회사명', '거래소코드', '회계년도', '기업규모코드', "현금비율", "순운전자본비율", '매출액순이익률', "총자본사업이익률", "부가가치율"]]

In [385]:
# 대기업 30개 대기업 외 30개 기업 랜덤 추출(가입시점)

total_d1 = d1.dropna()

# 기업규모코드가 10인 데이터 추출
scale_10_ = total_d1.loc[total_d1['기업규모코드'] == 10]

# 기업규모코드가 10이 아닌 데이터 추출
scale_N10_ = total_d1.loc[total_d1['기업규모코드'] != 10]

# scale_10에서 랜덤하게 30개 추출
scale_10 = scale_10_.sample(n=30, random_state=12)

# scale_N10에서 랜덤하게 30개 추출
scale_N10 = scale_N10_.sample(n=30, random_state=12)

# 추출한 기업들을 하나로 합치기
selected_companies_1 = pd.concat([scale_10, scale_N10])

In [386]:
# 대기업 30개 대기업 외 30개 기업 랜덤 추출(현재시점)

total_d2 = d2.dropna()

# 기업규모코드가 10인 데이터 추출
scale_10_ = total_d2.loc[total_d2['기업규모코드'] == 10]

# 기업규모코드가 10이 아닌 데이터 추출
scale_N10_ = total_d2.loc[total_d2['기업규모코드'] != 10]

# scale_10에서 랜덤하게 30개 추출
scale_10 = scale_10_.sample(n=30, random_state=20)

# scale_N10에서 랜덤하게 30개 추출
scale_N10 = scale_N10_.sample(n=30, random_state=45)

# 추출한 기업들을 하나로 합치기
selected_companies_2 = pd.concat([scale_10, scale_N10])

In [387]:
# 랜덤 추출된 기업 데이터 인덱스 리셋
selected_companies_1 = selected_companies_1.reset_index(drop=True)
selected_companies_2 = selected_companies_2.reset_index(drop=True)

In [388]:
# 결측치 제거후 인덱스 리셋
d1 = d1.dropna().reset_index(drop=True)
d2 = d2.dropna().reset_index(drop=True)

In [389]:
# 10분할 점수 부여 기준(가입시점)

columns = d1.columns.tolist()[4:]

# 각 열에서 상위 10개의 데이터를 추출하는 for문을 작성합니다.
result = []
for col in columns:
    data = d1[col].sort_values(ascending=False).reset_index(drop=True).loc[[14, 30, 45, 60, 76, 91, 107, 122, 138, 153]]
    result.append(data)

# 추출한 결과를 데이터프레임으로 합쳐줍니다.
d1_top10 = pd.concat(result, axis=1)

# 데이터프레임을 출력합니다.
d1_top10


,현금비율,순운전자본비율,매출액순이익률,총자본사업이익률,부가가치율
14,42.669970,192.873323,13.207272,5.068999,100.000000
30,31.176770,140.669798,9.015971,3.454742,99.702030
45,21.404190,101.134585,7.046674,2.388860,97.680311
60,17.256386,79.626484,5.512844,1.971831,92.929144
76,14.434573,53.978369,3.686783,1.407884,90.544267
91,11.581141,31.368602,2.305742,1.017760,86.705140
107,8.319922,13.481653,0.451564,0.374696,83.595973
122,7.001685,-9.435525,-1.243232,-0.463690,79.727531
138,4.108714,-51.952946,-7.794690,-4.272414,73.089482
153,0.962602,-2871.530690,-1695.489900,-324.956135,-117.818673


In [413]:
# 현금비율의 등급(가입시점)
feature_1 = pd.DataFrame({
    '현금비율': [x for x in selected_companies_1['현금비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 42.669970:
        return 1
    elif value >= 31.176770:
        return 2
    elif value >= 21.404190:
        return 3
    elif value >= 17.256386:
        return 4
    elif value >= 14.434573:
        return 5
    elif value >= 11.581141:
        return 6
    elif value >= 8.319922:
        return 7
    elif value >= 7.001685:
        return 8
    elif value >= 4.108714:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_1['등급'] = feature_1['현금비율'].apply(get_grade)

grade1 = feature_1['등급'].sum()
grade1


310

In [391]:
# 순운전자본비율의 등급(가입시점)
feature_2 = pd.DataFrame({
    '순운전자본비율': [x for x in selected_companies_1['순운전자본비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 192.873323:
        return 1
    elif value >= 140.669798:
        return 2
    elif value >= 101.134585:
        return 3
    elif value >= 79.626484:
        return 4
    elif value >= 53.978369:
        return 5
    elif value >= 31.368602:
        return 6
    elif value >= 13.481653:
        return 7
    elif value >= -9.435525:
        return 8
    elif value >= -51.952946:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_2['등급'] = feature_2['순운전자본비율'].apply(get_grade)

grade2 = feature_2['등급'].sum()
grade2


305

In [392]:
# 매출액순이익률의 등급(가입시점)
feature_3 = pd.DataFrame({
    '매출액순이익률': [x for x in selected_companies_1['매출액순이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 13.207272:
        return 1
    elif value >= 9.015971:
        return 2
    elif value >= 7.046674:
        return 3
    elif value >= 5.512844:
        return 4
    elif value >= 3.686783:
        return 5
    elif value >= 2.305742:
        return 6
    elif value >= 0.451564:
        return 7
    elif value >= -1.243232:
        return 8
    elif value >= -7.794690:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_3['등급'] = feature_3['매출액순이익률'].apply(get_grade)

# 결과 출력
grade3 = feature_3['등급'].sum()
grade3

302

In [393]:
# 총자본사업이익률의 등급(가입시점)
feature_4 = pd.DataFrame({
    '총자본사업이익률': [x for x in selected_companies_1['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 5.068999:
        return 1
    elif value >= 3.454742:
        return 2
    elif value >= 2.388860:
        return 3
    elif value >= 1.971831:
        return 4
    elif value >= 1.407884:
        return 5
    elif value >= 1.017760:
        return 6
    elif value >= 0.374696:
        return 7
    elif value >= -0.463690:
        return 8
    elif value >= -4.272414:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_4['등급'] = feature_4['총자본사업이익률'].apply(get_grade)

grade4 = feature_4['등급'].sum()
grade4

314

In [414]:
# 부가가치율의 등급(가입시점)
feature_5 = pd.DataFrame({
    '부가가치율': [x for x in selected_companies_1['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 100.000000:
        return 1
    elif value >= 99.702030:
        return 2
    elif value >= 97.680311:
        return 3
    elif value >= 92.929144:
        return 4
    elif value >= 90.544267:
        return 5
    elif value >= 86.705140:
        return 6
    elif value >= 83.595973:
        return 7
    elif value >= 79.727531:
        return 8
    elif value >= 73.089482:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature_5['등급'] = feature_5['부가가치율'].apply(get_grade)

grade5 = feature_5['등급'].sum()
grade5

597

### 가입시점 펀드위험내부지수

In [395]:
join = (grade1 + grade2 + grade3 + grade4 + grade5) * 50/3000
join.round()

30.0

In [396]:
# 10분할 점수 부여 기준(현재시점)

columns = d2.columns.tolist()[4:]

# 각 열에서 상위 10개의 데이터를 추출하는 for문을 작성합니다.
result = []
for col in columns:
    data = d2[col].sort_values(ascending=False).reset_index(drop=True).loc[[14, 30, 45, 60, 76, 91, 107, 122, 138, 153]]
    result.append(data)

# 추출한 결과를 데이터프레임으로 합쳐줍니다.
d2_top10 = pd.concat(result, axis=1)

# 데이터프레임을 출력합니다.
d2_top10

,현금비율,순운전자본비율,매출액순이익률,총자본사업이익률,부가가치율
14,245.419328,30.738807,13.937082,141.331309,101.818218
30,116.962658,17.265843,9.174021,105.086744,100.597142
45,79.685660,9.865081,6.667611,100.238322,100.026696
60,49.161383,6.355522,5.197036,98.771828,100.000000
76,21.334342,2.676220,3.631935,93.060983,99.946618
91,5.250606,0.545803,2.637603,80.476820,99.521083
107,-15.049444,-2.275697,1.366735,59.841530,99.062290
122,-85.682368,-9.067939,-0.086659,26.455501,98.288158
138,-292.014915,-27.798076,-5.619149,-9.279052,97.494356
153,-9832.346071,-2843.537475,-2887.053813,-5884.794157,-11.051269


In [397]:
# 현금비율의 등급(현재시점)
feature1 = pd.DataFrame({
    '현금비율': [x for x in selected_companies_2['현금비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 245.419328:
        return 1
    elif value >= 116.962658:
        return 2
    elif value >= 79.685660:
        return 3
    elif value >= 49.161383:
        return 4
    elif value >= 21.334342:
        return 5
    elif value >= 5.250606:
        return 6
    elif value >= -15.049444:
        return 7
    elif value >= -85.682368:
        return 8
    elif value >= -292.014915:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature1['등급'] = feature1['현금비율'].apply(get_grade)

grade_1 = feature1['등급'].sum()
grade_1


359

In [398]:
# 순운전자본비율의 등급(현재시점)
feature2 = pd.DataFrame({
    '순운전자본비율': [x for x in selected_companies_2['순운전자본비율']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 30.738807:
        return 1
    elif value >= 17.265843:
        return 2
    elif value >= 9.865081:
        return 3
    elif value >= 6.355522:
        return 4
    elif value >= 2.676220:
        return 5
    elif value >= 0.545803:
        return 6
    elif value >= -2.275697:
        return 7
    elif value >= -9.067939:
        return 8
    elif value >= -27.798076:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature2['등급'] = feature2['순운전자본비율'].apply(get_grade)

grade_2 = feature2['등급'].sum()
grade_2


357

In [399]:
# 매출액순이익률의 등급(현재시점)
feature3 = pd.DataFrame({
    '매출액순이익률': [x for x in selected_companies_2['매출액순이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 13.937082:
        return 1
    elif value >= 9.174021:
        return 2
    elif value >= 6.667611:
        return 3
    elif value >= 5.197036:
        return 4
    elif value >= 3.631935:
        return 5
    elif value >= 2.637603:
        return 6
    elif value >= 1.366735:
        return 7
    elif value >= -0.086659:
        return 8
    elif value >= -5.619149:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature3['등급'] = feature3['매출액순이익률'].apply(get_grade)

# 결과 출력
grade_3 = feature3['등급'].sum()
grade_3

348

In [400]:
# 총자본사업이익률의 등급(현재시점)
feature4 = pd.DataFrame({
    '총자본사업이익률': [x for x in selected_companies_2['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 141.331309:
        return 1
    elif value >= 105.086744:
        return 2
    elif value >= 100.238322:
        return 3
    elif value >= 98.771828:
        return 4
    elif value >= 93.060983:
        return 5
    elif value >= 80.476820:
        return 6
    elif value >= 59.841530:
        return 7
    elif value >= 26.455501:
        return 8
    elif value >= -9.279052:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature4['등급'] = feature4['총자본사업이익률'].apply(get_grade)

grade_4 = feature4['등급'].sum()
grade_4

367

In [401]:
# 부가가치율의 등급(현재시점)
feature5 = pd.DataFrame({
    '부가가치율': [x for x in selected_companies_2['총자본사업이익률']]
})

# 등급 매핑을 위한 함수 생성
def get_grade(value):
    if value >= 101.818218:
        return 1
    elif value >= 100.597142:
        return 2
    elif value >= 100.026696:
        return 3
    elif value >= 100.000000:
        return 4
    elif value >= 99.946618:
        return 5
    elif value >= 99.521083:
        return 6
    elif value >= 99.062290:
        return 7
    elif value >= 98.288158:
        return 8
    elif value >= 97.494356:
        return 9
    else:
        return 10

# apply 함수를 사용하여 등급 컬럼 추가
feature5['등급'] = feature5['부가가치율'].apply(get_grade)

grade_5 = feature5['등급'].sum()
grade_5

457

### 현재 펀드위험내부지수

In [402]:
present = (grade_1 + grade_2 + grade_3 + grade_4 + grade_5) * 50/3000
present.round()

31.0

In [412]:
# 외부지수
Exchange_Rate_2019 = pd.read_csv("./공모전파일/2019 일별 환율.csv")
Exchange_Rate_2020 = pd.read_csv("./공모전파일/2020 일별 환율.csv")
Exchange_Rate_2019 = Exchange_Rate_2019.transpose().iloc[4 :]
Exchange_Rate_2019.columns = ['Exchange Rate']
Exchange_Rate_2019['Exchange Rate'] = Exchange_Rate_2019['Exchange Rate'].str.replace(',', '').astype(float)
Exchange_Rate_2019['Change'] = Exchange_Rate_2019['Exchange Rate'].diff()
volatility = np.std(Exchange_Rate_2019['Change'])
volatility

4.078599278367662